# Tutorial: Creating TF Records

Last updated: 09-sep-2021 by
Cristobal Donoso

In [1]:
cd ../..

/home/users/cdonoso/astromer/ASTROMER


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import sys,os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from core.masking import get_padding_mask
from core.astromer import get_ASTROMER, train
from core.data  import (create_dataset, 
                        pretraining_records)

# This tutorial runs only in early mode. For the optimized code use train.py
# OTHERWISE you can comment the next ljupyine and reset the kernel before the finetuning
tf.config.run_functions_eagerly(False)

%load_ext autoreload
%autoreload 2

In [8]:
source = './data/raw_data/alcock/LCs/' # lightcurves folder
metadata = './data/raw_data/alcock/alcock_dataset.dat' # metadata file
name = 'alcock'

### Metadata frame sample

In [9]:
meta = pd.read_csv(metadata)
meta = meta[meta['Class'] != 'UNK']
meta = meta[meta['Class'] != 'std']
# meta = meta[meta['N'] >= 20]
meta['Band'] = tf.ones(meta.shape[0])
meta = meta.rename(columns={'Unnamed: 0':'ID', 'Path_R':'Path'})

In [10]:
meta.sample()

,ID,Class,rPer,bPer,Path_B,Path,Band
21421,9.5606.838,RRc,0.35302,0.35301,./LCs/B/9.5606.838.dat,./LCs/R/9.5606.838.dat,1.0


### Lightcurve frame sample

In [11]:
lc_df = pd.read_csv(os.path.join(source,  
                                 meta['Path'].sample(1).values[0].split('/')[-1]))
lc_df.head()

,mjd,mag,err
0,48885.58594,-4.729,0.056
1,48908.42578,-4.130,0.092
2,48909.54688,-4.382,0.080
3,48918.69141,-4.167,0.051
4,48919.50000,-4.106,0.097


### Creating training partitions from .csv to .record 

In [12]:
%%time

for fold_n in range(3): 
    test_meta  = pd.concat([frame.sample(n=100) for g, frame in meta.groupby('Class')])
    train_meta = meta[~meta['ID'].isin(test_meta['ID'])]
    print(test_meta.shape, train_meta.shape)
        
    for nsamples in [20, 50, 100, 500, 0]:  
        if nsamples == 0:
            partial_meta = train_meta
            target = './data/records/{}/fold_{}/{}'.format(name, fold_n, name)
        else:
            partial_meta = pd.concat([frame.sample(n=nsamples) for c, frame in train_meta.groupby('Class')])
            target = './data/records/{}/fold_{}/{}_{}'.format(name, fold_n, name, nsamples)

        create_dataset(partial_meta, source, target, max_lcs_per_record=20000, 
                       n_jobs=7, subsets_frac=(0.8, 0.2), test_subset=test_meta)

(600, 7) (20294, 7)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:29<00:00,  4.97s/it]


(600, 7) (20294, 7)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:29<00:00,  4.96s/it]


(600, 7) (20294, 7)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:29<00:00,  4.96s/it]

CPU times: user 2min 43s, sys: 3.28 s, total: 2min 47s
Wall time: 3min


### JUST ONE DATASET

In [127]:
train_meta.shape
test_meta.shape
name

'new_ogle'

In [128]:
for fold_n in range(3):
    target = './data/records/{}/fold_{}/{}'.format(name, fold_n, name)

    create_dataset(train_meta, source, target, max_lcs_per_record=20000, 
                   n_jobs=7, subsets_frac=(0.8, 0.2), test_subset=test_meta,
                  names=['mjd', 'mag', 'err'], delim_whitespace=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:56<00:00, 17.60s/it]


In [17]:
root = './data/records/new_ogle/'
for fold_n in range(3):
    fold_f = os.path.join(root, 'fold_{}'.format(fold_n))
    for source in os.listdir(fold_f):
        target = source.replace('new_', '')
        os.rename(os.path.join(fold_f, source), os.path.join(fold_f, target))